In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_data = pd.read_csv('../input/train.csv')
test_data= pd.read_csv('../input/test.csv')

In [ ]:
#convert categorical values into numeric 
cleanup = { 'Sex': {'male':1, 'female':0}, 'Embarked':{'S':0, 'C':1, 'Q':2}}
train_data.replace(cleanup, inplace= True)
test_data.replace(cleanup, inplace = True)

In [ ]:
#Keep only relevant features (name and passenger ID are not relevant)
train_data= train_data[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Fare', 'Embarked']]
test_data= test_data[[ 'Pclass', 'Sex', 'Age', 'SibSp', 'Fare', 'Embarked']]

In [ ]:
#Drop null and missing values
train_data = train_data.dropna()
test_data = test_data.dropna()

In [ ]:
#Scale all attribute values to between 0 and 1
min_max=MinMaxScaler()
Train_data_scaled =min_max.fit_transform(train_data[['Survived','Pclass', 'Sex', 'Age', 'SibSp', 'Fare', 'Embarked']])
Test_data_scaled =min_max.fit_transform(test_data[['Pclass', 'Sex', 'Age', 'SibSp', 'Fare', 'Embarked']])
print(Train_data_scaled.shape) #Number of rows in training set

print(Test_data_scaled.shape) #number of rows in test set 

Xtrain = []
Ytrain = []
for i in range(Train_data_scaled.shape[0]):
    Ytrain.append(Train_data_scaled[i][0])
    Xtrain.append(list(Train_data_scaled[i][1:]))
Xtrain = np.array(Xtrain)
Ytrain = np.array(Ytrain)
Train_data_Xtf = tf.convert_to_tensor(Xtrain, np.float32)
Train_data_Ytf = tf.convert_to_tensor(Ytrain, np.float32)

Test_data_Xtf = tf.convert_to_tensor(Test_data_scaled, np.float32)
Test_data_Xtf.shape

In [ ]:
learning_rate = 0.001
training_epochs = 15 #Number of iterations
batch_size = 50 #50 samples per iteration
display_step =1
hidden1 =16  #16 neurons in first hidden layer
hidden2 = 16 
inputlayer = 6 #6 inputs - Pclass, Sex, Age, sibsp, Fare, Embarked
outputlayer = 1 #1 output - survived 


In [ ]:
#Placeholders for X - input and Y - output
X = tf.placeholder("float", [None, inputlayer])
Y = tf.placeholder("float", [None, outputlayer])

In [ ]:
#Randomly initialized weights and biases
weights = {
    'h1': tf.Variable(tf.random_normal([inputlayer, hidden1])),
    'h2': tf.Variable(tf.random_normal([hidden1, hidden2])),
    'out': tf.Variable(tf.random_normal([hidden2, outputlayer]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([hidden1])),
    'b2': tf.Variable(tf.random_normal([hidden2])),
    'out': tf.Variable(tf.random_normal([outputlayer]))
}


In [ ]:
def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
    logits=multilayer_perceptron(X), labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Initializing the variables
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(int(Train_data_Xtf.get_shape()[0])/batch_size)
        # Loop over all batches
        print(total_batch)
        for i in range(total_batch):
            batch_x = Train_data_Xtf[i*total_batch: total_batch+i*total_batch+1].eval()
            batch_y = Train_data_Ytf[i*total_batch: total_batch+i*total_batch+1].eval()
            batch_x=batch_x.reshape(-1,6)
            batch_y=batch_y.reshape(-1,1)
            
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,
                                                            Y: batch_y})
            
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    pred = tf.nn.softmax(multilayer_perceptron(X))  # Apply softmax to logits
    predicted_class = tf.greater(pred,0.5)
    correct = tf.equal(predicted_class, tf.equal(Y,1.0))
    accuracy = tf.reduce_mean( tf.cast(correct, 'float') )
    test_acc = sess.run(accuracy, feed_dict={X: Test_data_Xtf.eval().reshape(-1,6), Y: Train_data_Ytf[:331].eval().reshape(-1,1)})
    print ("Test accuracy: %.3f" % (test_acc))
                        
                                      


In [ ]:
Test_data_Xtf